
# BrainCell 快速建模教程


## 一、基础元件建模
基础元件是构建神经元模型的核心单元，包含``离子``和 ``离子通道``，直接决定神经元电生理特性，如动作电位产生、膜电流变化等。


### 离子(``Ion``)
**作用**：
离子，如 Na⁺、K⁺、Ca²⁺等，是神经元电活动的核心载体，其跨膜流动产生电流，驱动膜电位变化。
**核心功能**：
- 定义离子基本属性：名称、大小、通道、离子浓度、反转电位等。
- 支持与离子通道关联，建立通道与离子的选择性通透关系，比如 Na⁺ 通道仅允许 Na⁺ 通过。


### 离子通道(``IonChannel``)
**作用**：
模拟细胞膜上选择性通透离子的蛋白结构，控制离子流动。
**核心功能**：
- 定义通道特性：关联离子类型、最大电导、门控动力学等。
- 支持 Hodgkin-Huxley 型动力学。


## 二、房室建模
房室是模拟神经元空间结构的基本构件，可独立或组合使用，实现从单区域到全神经元的动态模拟。


### 单房室(``SingleCompartment``)
**作用**：
简化模拟神经元局部区域，快速验证离子通道组合的电生理特性。
**核心功能**：
- 定义房室物理参数：名称、大小、膜电容、阈值电压等。
- 支持添加离子通道，构建膜与通道的耦合系统，计算跨膜电流和膜电位变化。


### 多房室(``MultiCompartment``)
**作用**：
模拟神经元复杂空间结构，实现了具有多个连接房室的详细神经元模型，研究电生理信号在神经元内的空间传播。
**核心功能**：
- 构建房室网络：指定房室间连接关系。
- 支持参数设置：不同房室可定义独立的尺寸与电学特征，模拟真实神经元的差异性。


### HH 模型(``HHTypedNeuron``)
**作用**：
基于经典 Hodgkin-Huxley 理论的单房室模型，建模动作电位产生的完整过程。
**核心优势**：
- 构建经典模型：揭示动作电位产生与传播的电生理机制。
- 支持参数设置：支持修改相关参数，用于验证不同离子机制对神经元兴奋性的影响。


## 三、微分方程建模
通过``DiffEqState``和``DiffEqModule``实现神经元动态的数学描述，将生理过程转化为状态变量演化的微分方程系统。


### 状态变量管理(``DiffEqState``)
**作用**：
统一管理系统状态变量，如膜电位、通道门控变量等，提供标准化的初始化、存储和访问接口。
**核心功能**：
- 定义多维度状态变量：支持微分方程，适配单神经元或网络模拟。
- 兼容[JAX](https://github.com/jax-ml/jax)  数组：无缝对接[JAX](https://github.com/jax-ml/jax) 自动微分和向量计算，提升大规模模拟效率。


### 微分方程定义(``DiffEqModule``)
**作用**：
定义状态变量的演化规则，描述膜电位、门控变量随时间的变化规律。
**核心功能**：
- 实现梯度方法：输入当前状态和时间，返回各变量的导数。


## 快速上手建议
1. **从单房室开始**：使用``HHNeuron`` 类快速复现动作电位，熟悉基础工作流。
2. **模块化扩展**：逐步添加自定义离子通道或多房室结构，利用``braincell``的组件化设计降低复杂度。
3. **结合[JAX](https://github.com/jax-ml/jax)加速**：通过 ``jit``优化模拟性能，尤其适用于大规模网络场景。

通过分层建模框架，``braincell`` 支持从离子通道到神经元网络的全尺度模拟，兼顾电生理精确性与计算效率。
下面，我们会向你介绍一个例子，方便你更好的理解``braincell``在实际建模中的应用。


## 四、实例：兴奋性-抑制性神经元网络（E-I网络）的模拟与分析

以下将通过一个完整实例，展示如何基于前述基础元件和房室模型构建复杂网络，实现从单细胞到群体动态的层级建模。

### 准备工作
导入所需库，以下库在后续教程中也会经常调用：

```python
import brainunit as u  # 神经科学单位系统
import matplotlib.pyplot as plt  # 可视化工具
import brainstate  # 网络动态管理
import braincell  # 核心建模库
```


### 核心参数定义
定义神经元物理属性与网络规模参数：

```python
# 1.1 神经元电生理参数
V_th = -20. * u.mV  # 动作电位发放阈值

# 膜面积
area = 20000 * u.um **2
area = area.in_unit(u.cm** 2)  # 转换为平方厘米

# 膜电容：总电容 = 比电容 × 面积
Cm = (1 * u.uF * u.cm ** -2) * area  # 单位：pF
```


### 构建 HH 单神经元模型
基于 ``SingleCompartment`` 类，整合钠通道、钾通道和漏电流：

```python
class HH(braincell.SingleCompartment):
    def __init__(self, in_size):
        # 初始化单房室
        # 积分方法采用独立指数欧拉法
        super().__init__(in_size, C=Cm, solver='ind_exp_euler', V_th=V_th)

        # 2.1 钠离子通道
        self.na = braincell.ion.SodiumFixed(in_size, E=50. * u.mV)  # 钠反转电位
        self.na.add_elem(
            # 最大电导 = 比电导 × 膜面积
            INa=braincell.channel.INa_TM1991(
                in_size,
                g_max=(100. * u.mS * u.cm **-2) * area,
                V_sh=-63. * u.mV  # 电压偏移
            )
        )

        # 2.2 钾离子通道
        self.k = braincell.ion.PotassiumFixed(in_size, E=-90. * u.mV)  # 钾反转电位
        self.k.add_elem(
            IK=braincell.channel.IK_TM1991(
                in_size,
                g_max=(30. * u.mS * u.cm** -2) * area,
                V_sh=-63. * u.mV
            )
        )

        # 2.3 漏电流
        self.IL = braincell.channel.IL(
            in_size,
            E=-60. * u.mV,  # 漏电流反转电位
            g_max=(5. * u.nS * u.cm **-2) * area
        )
```


### 构建 E-I 网络
组合 HH 神经元，定义兴奋性和抑制性突触连接：

```python
class EINet(brainstate.nn.DynamicsGroup):
    def __init__(self):
        super().__init__()
        # 网络规模：3200 兴奋性 + 800 抑制性 神经元
        self.n_exc = 3200
        self.n_inh = 800
        self.total = self.n_exc + self.n_inh

        # 初始化神经元群体
        self.neurons = HH(self.total)

        # 兴奋性突触投射
        self.exc_proj = brainstate.nn.AlignPostProj(
            # 连接规则：E 神经元以 2% 概率连接所有神经元
            comm=brainstate.nn.EventFixedProb(
                pre_size=self.n_exc,
                post_size=self.total,
                conn_num=0.02,  # 连接概率
                conn_weight=6. * u.nS  # 突触电导
            ),
            # 突触动力学：典型兴奋性突触
            syn=brainstate.nn.Expon(self.total, tau=5. * u.ms),
            # 突触后效应
            out=brainstate.nn.COBA(E=0. * u.mV),
            post=self.neurons
        )

        # 抑制性突触投射
        self.inh_proj = brainstate.nn.AlignPostProj(
            # 连接规则：I 神经元以 2% 概率连接所有神经元
            comm=brainstate.nn.EventFixedProb(
                pre_size=self.n_inh,
                post_size=self.total,
                conn_num=0.02,
                conn_weight=67. * u.nS
            ),
            # 突触动力学：指数衰减,典型抑制性突触
            syn=brainstate.nn.Expon(self.total, tau=10. * u.ms),
            # 突触后效应
            out=brainstate.nn.COBA(E=-80. * u.mV),
            post=self.neurons
        )

    def update(self, t):
        # 网络动态更新规则
        with brainstate.environ.context(t=t):
            # 获取当前时刻的 spike 信号
            spikes = self.neurons.spike.value

            # 兴奋性神经元的 spike 驱动兴奋性突触
            self.exc_proj(spikes[:self.n_exc])  # 前 n_exc 个为 E 神经元

            # 抑制性神经元的 spike 驱动抑制性突触
            self.inh_proj(spikes[self.n_exc:])  # 剩余为 I 神经元

            # 更新神经元状态
            new_spikes = self.neurons(0. * u.nA)
            return new_spikes
```


### 运行网络模拟
配置仿真参数并执行模拟：

```python
# 初始化网络
net = EINet()
brainstate.nn.init_all_states(net)  # 初始化所有神经元和突触状态

# 设置仿真环境:时间步长 0.1 ms，高精度捕获 spike
with brainstate.environ.context(dt=0.1 * u.ms):
    # 生成时间序列
    times = u.math.arange(0. * u.ms, 100. * u.ms, brainstate.environ.get_dt())

    # 运行模拟（
    spikes = brainstate.compile.for_loop(
        net.update,
        times,
        pbar=brainstate.compile.ProgressBar(10)
    )
```


### 可视化网络动态
绘制spike 发放时间-神经元索引图：

```python
# 提取 spike 发生的时间和神经元索引
t_indices, n_indices = u.math.where(spikes)

# 绘制 raster plot
plt.figure(figsize=(10, 6))
plt.scatter(
    times[t_indices].to_decimal(u.ms),  # 横轴：时间
    n_indices,  # 纵轴：神经元索引
    s=1, color='k'
)
plt.xlabel('时间（ms）')
plt.ylabel('神经元索引')
plt.title('E-I 网络的 Spike 发放模式')
plt.show()
```
通过本实例，你可以直观地理解不同层级在建模中的实际应用。当然，如果你需要更为详细的教程，可以阅读tutorial部分，以及查看后面的的API文档。
